In [1]:
import pandas as pd

In [2]:
df_orig = pd.read_csv('dataset/pn.csv.m3.120408.trim', sep='\t', header=None, names=['word', 'polarity', 'temp1'])

In [3]:
df_nega = df_orig.query('polarity=="n"').copy()
df_nega

,word,polarity,temp1
46,２次感染,n,〜する（出来事）
47,２失点,n,〜する（出来事）
68,３連敗,n,〜する（出来事）
88,ＡＩＤＳ,n,〜である・になる（状態）客観
91,Ａ型肝炎,n,〜である・になる（状態）客観
...,...,...,...
13304,頽廃,n,〜する（出来事）
13305,顰蹙,n,〜する（行為）
13306,騙り,n,〜がある・高まる（存在・性質）
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観


In [4]:
# 読み仮名どうする
# 単純に読み仮名を振る → Pykaksi
# MeCabなら→読み仮名と発音を両方取得できる！
# MeCab使ってみるか


In [5]:
import MeCab

In [6]:
tagger = MeCab.Tagger()

In [7]:
print(tagger.parse("修学旅行に出かける"))

修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0
EOS



In [8]:
result = tagger.parse("修学旅行に出かける")
print(type(result))
print(result)

<class 'str'>
修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0
EOS



In [9]:
# EOSの手前を抽出
result = result.split('\nEOS')[0]

In [10]:
print(result)

修学	シューガク	シュウガク	修学	名詞-普通名詞-サ変可能			0
旅行	リョコー	リョコウ	旅行	名詞-普通名詞-サ変可能			0
に	ニ	ニ	に	助詞-格助詞			
出かける	デカケル	デカケル	出掛ける	動詞-一般	下一段-カ行	終止形-一般	0


In [11]:
[x.split('\t') for x in result.split('\n')]

[['修学', 'シューガク', 'シュウガク', '修学', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['旅行', 'リョコー', 'リョコウ', '旅行', '名詞-普通名詞-サ変可能', '', '', '0'],
 ['に', 'ニ', 'ニ', 'に', '助詞-格助詞', '', '', ''],
 ['出かける', 'デカケル', 'デカケル', '出掛ける', '動詞-一般', '下一段-カ行', '終止形-一般', '0']]

In [12]:
''.join([x.split('\t')[1] for x in result.split('\n')])

'シューガクリョコーニデカケル'

In [13]:
def extract_yomi(word):
    tagger = MeCab.Tagger()
    result = tagger.parse(word)
    # EOSの手前を抽出
    result = result.split('\nEOS')[0]
    yomi = ''.join([x.split('\t')[1] for x in result.split('\n')])
    return yomi

In [14]:
extract_yomi("子供の落書き帳")

'コドモノラクガキチョー'

In [15]:
list(df_nega['word'])[1000:1010]

['猿知恵', '艶聞', '遠回り', '遠島', '遠慮気味', '鉛中毒', '塩害', '塩辛さ', '塩素臭', '汚さ']

In [16]:
lis = list(df_nega['word'])[1000:1010]

In [17]:
[extract_yomi(word) for word in lis]

['サルチエ',
 'エンブン',
 'トーマワリ',
 'エントー',
 'エンリョキミ',
 'ナマリチュードク',
 'エンガイ',
 'シオカラサ',
 'エンソシュー',
 'キタナサ']

In [18]:
lis = list(df_nega['word'])

In [19]:
df_nega['pronunciation'] = [extract_yomi(word) for word in lis]
df_nega

,word,polarity,temp1,pronunciation
46,２次感染,n,〜する（出来事）,２ジカンセン
47,２失点,n,〜する（出来事）,２シッテン
68,３連敗,n,〜する（出来事）,３レンパイ
88,ＡＩＤＳ,n,〜である・になる（状態）客観,エイズ
91,Ａ型肝炎,n,〜である・になる（状態）客観,Ａガタカンエン
...,...,...,...,...
13304,頽廃,n,〜する（出来事）,タイハイ
13305,顰蹙,n,〜する（行為）,ヒンシュク
13306,騙り,n,〜がある・高まる（存在・性質）,カタリ
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観,ヒニクノタン


In [20]:
import pykakasi
kks = pykakasi.kakasi()

In [21]:
def extract_romaji(word):
    """
    入力として「修学旅行」と「シューガクリョコー」のどちらが良いか考え中
    """
    result = kks.convert(word)
    romaji = ''.join([item['hepburn'] for item in result])
    return romaji

In [22]:
df_nega['romaji'] = [extract_romaji(word) for word in lis]
df_nega

,word,polarity,temp1,pronunciation,romaji
46,２次感染,n,〜する（出来事）,２ジカンセン,2tsugikansen
47,２失点,n,〜する（出来事）,２シッテン,2shitten
68,３連敗,n,〜する（出来事）,３レンパイ,3renpai
88,ＡＩＤＳ,n,〜である・になる（状態）客観,エイズ,AIDS
91,Ａ型肝炎,n,〜である・になる（状態）客観,Ａガタカンエン,Akatakan'en
...,...,...,...,...,...
13304,頽廃,n,〜する（出来事）,タイハイ,taihai
13305,顰蹙,n,〜する（行為）,ヒンシュク,hinshuku
13306,騙り,n,〜がある・高まる（存在・性質）,カタリ,katari
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観,ヒニクノタン,hinikunotan


In [23]:
def extract_vowels(word):
    return ''.join([letter for letter in word if letter in 'aeiou'])

In [24]:
extract_vowels('linus')

'iu'

In [25]:
df_nega['vowels'] = [extract_vowels(word) for word in df_nega['romaji']]
df_nega

,word,polarity,temp1,pronunciation,romaji,vowels
46,２次感染,n,〜する（出来事）,２ジカンセン,2tsugikansen,uiae
47,２失点,n,〜する（出来事）,２シッテン,2shitten,ie
68,３連敗,n,〜する（出来事）,３レンパイ,3renpai,eai
88,ＡＩＤＳ,n,〜である・になる（状態）客観,エイズ,AIDS,
91,Ａ型肝炎,n,〜である・になる（状態）客観,Ａガタカンエン,Akatakan'en,aaae
...,...,...,...,...,...,...
13304,頽廃,n,〜する（出来事）,タイハイ,taihai,aiai
13305,顰蹙,n,〜する（行為）,ヒンシュク,hinshuku,iuu
13306,騙り,n,〜がある・高まる（存在・性質）,カタリ,katari,aai
13308,髀肉の嘆,n,〜である・になる（評価・感情）主観,ヒニクノタン,hinikunotan,iiuoa


* 顰蹙→iuu になっている。ンが考慮されていない。ただここでnを入れると「ナニヌネノ」がそのまま残ってしまうので、ンだけ考慮したい。
* 読みが異なるケースが稀にあるっぽい
  * 8182	生霊	n	〜である・になる（状態）客観	イキリョー	seirei	eiei 


In [26]:
df_nega.tail(30)

,word,polarity,temp1,pronunciation,romaji,vowels
13266,訃音,n,〜がある・高まる（存在・性質）,フイン,fuin,ui
13267,訃報,n,〜がある・高まる（存在・性質）,フホー,fuhou,uou
13268,訌争,n,〜がある・高まる（存在・性質）,コーソー,kousou,ouou
13269,訛り,n,〜がある・高まる（存在・性質）,ナマリ,namari,aai
13270,詭弁,n,〜である・になる（評価・感情）主観,キベン,kiben,ie
13271,諫止,n,〜する（行為）,カンシ,kanshi,ai
13272,謗法,n,〜である・になる（評価・感情）主観,ホーボー,houbou,ouou
13273,謫居,n,〜する（行為）,タッキョ,takkyo,ao
13274,譴責,n,〜がある・高まる（存在・性質）,ケンセキ,kenseki,eei
13276,貪欲,n,〜である・になる（評価・感情）主観,ドンヨク,donyoku,oou


In [27]:
df_nega.query("vowels=='ouou'").head(20)

,word,polarity,temp1,pronunciation,romaji,vowels
483,ご苦労,n,〜がある・高まる（存在・性質）,ゴクロー,gokurou,ouou
859,もうろく,n,〜する（出来事）,モーロク,mouroku,ouou
911,ろくろく,n,〜である・になる（評価・感情）主観,ロクロク,rokuroku,ouou
2935,凹凸,n,〜する（出来事）,オートツ,outotsu,ouou
2942,往生,n,〜する（出来事）,オージョー,oujou,ouou
2971,横行,n,〜する（出来事）,オーコー,oukou,ouou
2975,横暴,n,〜である・になる（評価・感情）主観,オーボー,oubou,ouou
2977,横領,n,〜する（行為）,オーリョー,ouryou,ouou
2998,臆病,n,〜である・になる（評価・感情）主観,オクビョー,okubyou,ouou
4241,共謀,n,〜する（行為）,キョーボー,kyoubou,ouou


In [28]:
df_nega.query("word=='生霊'")

,word,polarity,temp1,pronunciation,romaji,vowels
8182,生霊,n,〜である・になる（状態）客観,イキリョー,seirei,eiei
